In [6]:
lista_1 = [1, 2, 3, 4, 5]

for index, x in enumerate(lista_1):
    x = x * 2

print(lista_1)

[1, 2, 3, 4, 5]


In [19]:
import copy
from dataclasses import dataclass
from enum import Enum
from hmac import new
from typing import List


@dataclass
class States:
    class Node(Enum):
        REST = 1
        RESERVED = 2
        OCCUPIED = 3
        # SENDING = 4

    class Destiny(Enum):
        AVAILABLE = 1
        NOT_AVAILABLE = 2

    node: Node
    destinies: List[Destiny]


states = States(States.Node.OCCUPIED, [States.Destiny.AVAILABLE, States.Destiny.NOT_AVAILABLE])

print(states)

match states:
    case States(States.Node.OCCUPIED, destinies=destinies):
        print("REST")
        print(destinies)
        for index, destiny in enumerate(destinies):
            if destiny == States.Destiny.AVAILABLE:
                new_destinies = copy.deepcopy(destinies)
                new_destinies[index] = States.Destiny.NOT_AVAILABLE
                new_states = States(States.Node.REST, new_destinies)
            
            print("New states1: ", new_states)

        print("New states2: ", new_states)

print("States: ", states)
print("New states4: ", new_states)


States(node=<Node.OCCUPIED: 3>, destinies=[<Destiny.AVAILABLE: 1>, <Destiny.NOT_AVAILABLE: 2>])
REST
[<Destiny.AVAILABLE: 1>, <Destiny.NOT_AVAILABLE: 2>]
New states1:  States(node=<Node.REST: 1>, destinies=[<Destiny.NOT_AVAILABLE: 2>, <Destiny.NOT_AVAILABLE: 2>])
New states1:  States(node=<Node.REST: 1>, destinies=[<Destiny.NOT_AVAILABLE: 2>, <Destiny.NOT_AVAILABLE: 2>])
New states2:  States(node=<Node.REST: 1>, destinies=[<Destiny.NOT_AVAILABLE: 2>, <Destiny.NOT_AVAILABLE: 2>])
States:  States(node=<Node.OCCUPIED: 3>, destinies=[<Destiny.AVAILABLE: 1>, <Destiny.NOT_AVAILABLE: 2>])
New states4:  States(node=<Node.REST: 1>, destinies=[<Destiny.NOT_AVAILABLE: 2>, <Destiny.NOT_AVAILABLE: 2>])


In [14]:
from calendar import c
from typing import TypeVar, Generic, List, Dict
from enum import Enum

class TestEnum(Enum):
    A = 1
    B = 2
    C = 3

T = TypeVar("T")


class CounterController(Generic[T]):
    def __init__(self, counters_indexes: List[T]):
        self.counter_indexes = counters_indexes
        self.counters: Dict[T, int] = {}

        for counter_index in self.counter_indexes:
            self.counters[counter_index] = 0

    def increment(self, counter_index: T):
        self.counters[counter_index] += 1


counter = CounterController(TestEnum)

counter.increment(TestEnum.A)

# counter.increment(TestEnum.B)

print(counter.counters)

{<TestEnum.A: 1>: 1, <TestEnum.B: 2>: 0, <TestEnum.C: 3>: 0}


In [57]:
# Delete repeated results in the results folder

import datetime
import os
import subprocess


results_folder = "../data/simulations/results"

results_files = []

results_files_checksum = []

for file in os.listdir(results_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(results_folder, file)
        results_files.append(file_path)

results_files.sort()

for file_dir in results_files:
    checksum_process = subprocess.run(
        ["openssl", "dgst", f"-SHA1", file_dir],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
    )
    results_files_checksum.append(checksum_process.stdout.split(" ")[-1].split("\n")[0])

results_files_to_delete = [False for _ in results_files_checksum]

for index, checksum in enumerate(results_files_checksum):
    if results_files_to_delete[index]:
        continue
    if index == len(results_files_checksum) - 1:
        break
    for index2, checksum2 in enumerate(results_files_checksum[index + 1 :], index + 1):
        if checksum == checksum2:
            results_files_to_delete[index2] = True

print("Summaries:")
for index, file in enumerate(results_files):
    print(f"{file: <70s} - {results_files_checksum[index]} - {results_files_to_delete[index]}")

time_of_check = datetime.datetime.now()

print("Checking date:")
print(time_of_check)

Summaries:
../data/simulations/results/01-02-2024_09-45-21.csv                    - adf1436f746c71997ca32c7367a29147c037c77c - False
../data/simulations/results/20-01-2024_12-00-00.csv                    - e03e7646fb50d9aa50f7abba1003155136a9676b - False
../data/simulations/results/25-01-2024_12-22-56.csv                    - 94977d632eb289b32b2cd41861320d550d2de8cd - False
../data/simulations/results/25-01-2024_13-42-25.csv                    - fe7b9b64733e49066a9ee8a66f6c4bb02357c1c3 - False
../data/simulations/results/25-01-2024_14-01-13.csv                    - 901a9c01cd75dc02dd1eaecdafdd3aed02ade734 - False
../data/simulations/results/30-01-2024_13-16-19.csv                    - a75d1a25fa225ce8fdce22b320ae3dcdefad8d6c - False
../data/simulations/results/31-01-2024_13-19-55.csv                    - 46086a7ddc00815ca986030fcbaaa6a4ac2f5a15 - False
../data/simulations/results/31-01-2024_13-21-16.csv                    - ff2ad37954ef2a55d5dbf8ee28ff5b78ad7e6426 - False
../data/simul

In [58]:


actual_time = datetime.datetime.now()

print("Checking date")
print(f"Actual date: {actual_time}")
print(f"Last checking date: {time_of_check}")
print(f"Elapsed time: {(actual_time - time_of_check)}")

if actual_time - time_of_check > datetime.timedelta(minutes=0.5):
    raise Exception("Last files check was more than 30 seconds ago")

print("Are you sure you want to delete the files? (y/n)", flush=True)
answer = input()

if answer == "y":
    print("Confirm delete with a123", flush=True)
    answer = input()

    if answer == "a123":
        for index, delete in enumerate(results_files_to_delete):
            if delete:
                os.remove(results_files[index])


Checking date
Actual date: 2024-02-01 11:00:46.213675
Last checking date: 2024-02-01 11:00:44.113611
Elapsed time: 0:00:02.100064
Are you sure you want to delete the files? (y/n)
Confirm delete with a123


In [4]:
results = {
    "asdf": {0: 34, 1: 23, 2: 12},
    "qwer": {0: 12, 2: 34},
    "zxcv": {0: 34, 1: 23},
}

objective = {0: [34, 12, 34], 1: [23, None, 23], 2: [12, 34, None]}

from collections import defaultdict

dd = {}

for d in results.values():  # you can list as many input dicts as you want here
    for key, value in d.items():
        if dd.get(key) is None:
            dd[key] = []
        dd[key].append(value)

print(dd)



{0: [34, 12, 34], 1: [23, 23], 2: [12, 34]}


In [ ]:
class TestEnum(Enum):
    A = 1
    B = 2
    C = 3
    
enum = TestEnum.A

print(enum)

print(enum.value)